In [7]:
import requests
from bs4 import BeautifulSoup
import re
import pandas as pd
import os 
import ast
import json
from requests.exceptions import ConnectionError

https://stackoverflow.com/questions/62335537/scraping-text-from-kickstarter-projects-return-nothing

In [3]:
base = "/Users/ivankoh/Downloads/kickstarter data/"
filepaths = [base + f for f in os.listdir(base) if f.endswith('.csv')]
df = pd.concat(map(pd.read_csv, filepaths))
df_1 = df.drop_duplicates(subset=['id'])
df_1['main_url'] = df_1['urls'].apply(lambda x: eval(x)['web']['project'])
print(len(df))
print(len(df_1))

392244
203510


/var/folders/dq/33rpc0cx4f54t0vx2cdcgczr0000gn/T/ipykernel_98894/4110031066.py:5: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_1['main_url'] = df_1['urls'].apply(lambda x: eval(x)['web']['project'])


In [4]:
s = requests.Session()
headers = {'User-Agent': 'Mozilla/5.0 (Windows NT 6.1; WOW64; rv:50.0) Gecko/20100101 Firefox/50.0'}
r = s.get("https://www.kickstarter.com", headers=headers)
print(r.cookies)
for cookie in r.cookies:
    print('cookie domain = ' + cookie.domain)
    print('cookie name = ' + cookie.name)
    print('cookie value = ' + cookie.value)
    print('*************************************')

<RequestsCookieJar[<Cookie __cf_bm=3_opx3uR5udjYYMyW8tn28ojaVPoqc_tq0CBmLa1PxI-1663600703-0-AaHV0X2YYhLCjHWmSDmRb9py+E5TKVhangSeAHbRgUhY4BD3Sc1Rj+IxJxXF1JgfEnC7kTyKU27IambsFwqu16Y= for .kickstarter.com/>, <Cookie _pxhd=82Y9H1hGpVlpPM1iIl7vB3VO7bzEnA024rSAnrritushJi/amIdsVKK/9FZrfxqSncojpJo/86pQsJE8VITdEQ==:T9sNNO/KsTDwY3iEyW51VVEu5mstqzImiRw1EVgDFpV2fV06V3OOLrl8TSgU837kHt35wBxmDjGzP8GwxT5uAu2WLQQH7m7VL1/aQoGFwmg= for www.kickstarter.com/>]>
cookie domain = .kickstarter.com
cookie name = __cf_bm
cookie value = 3_opx3uR5udjYYMyW8tn28ojaVPoqc_tq0CBmLa1PxI-1663600703-0-AaHV0X2YYhLCjHWmSDmRb9py+E5TKVhangSeAHbRgUhY4BD3Sc1Rj+IxJxXF1JgfEnC7kTyKU27IambsFwqu16Y=
*************************************
cookie domain = www.kickstarter.com
cookie name = _pxhd
cookie value = 82Y9H1hGpVlpPM1iIl7vB3VO7bzEnA024rSAnrritushJi/amIdsVKK/9FZrfxqSncojpJo/86pQsJE8VITdEQ==:T9sNNO/KsTDwY3iEyW51VVEu5mstqzImiRw1EVgDFpV2fV06V3OOLrl8TSgU837kHt35wBxmDjGzP8GwxT5uAu2WLQQH7m7VL1/aQoGFwmg=
********************************

In [13]:
def get_slug(url):
    return re.search('/projects/(.*)\?', url).group(1)

slugs = list(map(get_slug, df_1['main_url'].tolist()))
story_list = []
risk_list = []

# s = requests.Session()
# headers = {'User-Agent': 'Mozilla/5.0 (Windows NT 6.1; WOW64; rv:50.0) Gecko/20100101 Firefox/50.0'}
# r = s.get("https://www.kickstarter.com", headers=headers)
print(r.cookies)
soup = BeautifulSoup(r.text, 'html.parser')
xcsrf = soup.find("meta", {"name": "csrf-token"})["content"]

query = """
query Campaign($slug: String!) {
  project(slug: $slug) {
    risks
    story(assetWidth: 680)
  }
}"""

for slug in slugs:
    print(f"--------{slug}------")
    r = s.post("https://www.kickstarter.com/graph",
        headers= {
            'content-type': 'application/json',
            "x-csrf-token": xcsrf,
            'cookie': '_ksr_session=vtBM1mDYoeesrho8qrEIP4%2FDfdAE%2F1o6eqiAiH0bJvrzvCMvPjRRK9ZcW4LfK9exyg7fOEAwmVHR1xe%2ByUploeMFXGRFrBC5aP7GsrEAxgTYRBKJLY5RWumPfKJGwuRQDU4sxbEa0FJc2QdWzatr8ZGeDgukqQIpVbQrw3Bd67bQ3weTfOHCP9qcGVJXz7TAWAzOeYW%2BmGbMIw8jmnwNkn9UhpvyEb0rIRXoWCaynnuBIR9E3UbqlIF%2FWXJ0Sv1R0S2aq%2BmXiUOEJPD4i3LZWOCAOVc%3D--A%2BF2T2xdzsfMJ57%2B--f%2FlAFG2Cy4DHOp3TJ361ew%3D%3D',
            'User-Agent': 'Mozilla/5.0 (Macintosh; Intel Mac OS X 10_15_7) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/105.0.0.0 Safari/537.36'
        },
        json = {
            "operationName":"Campaign",
            "variables":{
                "slug": slug
            },
            "query": query
        })
    try:
        result = r.json()
        print("-------STORY--------")
        story_html = result["data"]["project"]["story"]
        soup = BeautifulSoup(story_html, 'html.parser')
        curr_story = []
        for i in soup.find_all('p'):
            print(i.get_text())
            curr_story.append(i.get_text())
        story_list.append(" ".join(curr_story) if curr_story else "")

        print("-------RISKS--------")
        print(result["data"]["project"]["risks"])
        risk_list.append(result["data"]["project"]["risks"] if result["data"]["project"]["risks"] else "")
    except:
        print(r)
        story_list.append("")
        risk_list.append("")
        continue

<RequestsCookieJar[<Cookie __cf_bm=lFJjk_0C2Nki3cYMnDeW953aMkv6xJi2ryk6RTgZg9A-1663600311-0-AcZYD7s2QqpGMHJHc+dO98gUKGfcLqfwq0GIpn5jZXaS5+SXqsNySXfh2Xb+5xNRJA/EXyFpEI9wABO69hqEUB8= for .kickstarter.com/>, <Cookie _ksr_session=ZG%2FcKY1B1lCcdRVKWzVaQrtuMGWpHd5VHB1gjB%2B8QQZkk43GNwzwcO6%2BZ4rhh4Mw1xezXmppm4E4XG%2FUz2W2QdQx8n4z7J5ScLBwA97u8pOSHNYZYmHBv6aYo%2FTvo1rJNbaRJuy9Zbd7chcsqRGthcDGBULRZRyicT5C0zDsRchkqxRe4Nz%2FF0S0V%2BoZOG74OSUWMVLvZKKB2vM6bJC7xAdnVfbuTgPyDLrvZtIGbnrjk5rOsnfzXMeU6XIIwEz4EWS9DEnHZBTG3oVlHkQUzB7qx0M%3D--crOYLil5D2Ac3hl%2B--Ob1EoVRc00TgAtuzjHE8iw%3D%3D for www.kickstarter.com/>, <Cookie _pxhd=JtQZv60WhVwuluui08JxjW9VwdNCAAc7W4cndO4j9J/mu3A6tDhfA4JRaQyS1QgkjrN4pgLJRSJC6o2FX7QmsA==:QP7A-4keDTn/4lm-zvcx1SU1r/AihxVP5gupVKpQLmc96c-cXSrzpmqkKd1DYHPv9gKdU6f8bFprCImtpF9etcrKSM9Rts6hNOWb3/wpvrM= for www.kickstarter.com/>, <Cookie lang=en for www.kickstarter.com/>, <Cookie last_page=https%3A%2F%2Fwww.kickstarter.com%2F for www.kickstarter.com/>, <Cookie optimizely_current_varia

KeyboardInterrupt: 

In [7]:
r

<Response [200]>